##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Trabajar con ClientData de TFF

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/working_with_client_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

La idea de un conjunto de datos que los clientes codifican (por ejemplo, usuarios) es esencial para el cálculo federado tal como se modela en TFF. TFF proporciona la interfaz [`tff.simulation.datasets.ClientData`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData) para abstraer este concepto y los conjuntos de datos que aloja TFF ([stackoverflow](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/stackoverflow), [shakespeare](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/shakespeare), [emnist](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist), [cifar100](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/cifar100) y [gldv2](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/gldv2)) implementan esta interfaz.

Si está trabajando en aprendizaje federado con su propio conjunto de datos, TFF le recomienda encarecidamente que implemente la interfaz `ClientData` o que use una de las funciones ayudantes de TFF para generar un `ClientData` que represente sus datos en el disco, por ejemplo, [`tff.simulation.datasets.ClientData.from_clients_and_fn`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData#from_clients_and_fn).

Como la mayoría de los ejemplos integrales de TFF comienzan con objetos `ClientData`, implementar la interfaz `ClientData` con su conjunto de datos personalizado hará que sea más fácil explorar el código existente escrito con TFF. Además, los `tf.data.Datasets` que construye `ClientData` se pueden iterar directamente para generar estructuras de arreglos `numpy`, por eso los objetos `ClientData` pueden usarse con cualquier marco de aprendizaje automático basado en Python antes de pasar a TFF.

Hay varios patrones que pueden facilitarle la vida si piensa aumentar sus simulaciones a muchas máquinas o si piensa implementarlas. A continuación, analizaremos algunas de las formas en que se puede usar `ClientData` y TFF para que nuestra experiencia de iteración a pequeña escala a experimentación a gran escala a implementación de producción sea lo más fluida posible.

## ¿Qué patrón debo usar para pasar ClientData a TFF?

Analizaremos dos usos de `ClientData` de TFF en profundidad; si usted entra en alguna de las dos categorías siguientes, claramente preferirá una sobre la otra. De lo contrario, es posible que necesite entender en más detalle las ventajas y las desventajas de cada uno para tomar una decisión más elaborada.

- Quiero iterar lo más rápido posible en una máquina local; no necesito poder aprovechar el tiempo de ejecución distribuido de TFF de forma fácil.

    - Quiere pasar `tf.data.Datasets` a TFF directamente.
    - Esto le permite programar de forma imperativa con objetos `tf.data.Dataset` y procesarlos arbitrariamente.
    - Proporciona más flexibilidad que la próxima opción; el enviar lógica a los clientes requiere que la lógica sea serializable.

- Quiero ejecutar mi cálculo federado en el tiempo de ejecución remoto de TFF o planeo hacerlo pronto.

    - En este caso, desea asignar la construcción y el preprocesamiento del conjunto de datos a los clientes.
    - Esto da como resultado que usted pase simplemente una lista de `client_ids` directamente a su cálculo federado.

- Enviar la construcción y el preprocesamiento de conjuntos de datos a los clientes evita cuellos de botella en la serialización y aumenta significativamente el rendimiento con cientos a miles de clientes.

In [ ]:
#@title Set up open-source environment
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow_federated

In [1]:
#@title Import packages
import collections
import time

import tensorflow as tf
import tensorflow_federated as tff

## Manipular un objeto ClientData

Comencemos por cargar y explorar `ClientData` de EMNIST de TFF:


In [2]:
client_data, _ = tff.simulation.datasets.emnist.load_data()

Al inspeccionar el primer conjunto de datos podemos ver qué tipo de ejemplos hay en `ClientData`.

In [3]:
first_client_id = client_data.client_ids[0]
first_client_dataset = client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)
# This information is also available as a `ClientData` property:
assert client_data.element_type_structure == first_client_dataset.element_spec

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])


Tenga en cuenta que el conjunto de datos produce objetos `collections.OrderedDict` que tienen `pixels` y claves `label`, donde píxeles es un tensor con forma `[28, 28]`. Supongamos que queremos aplanar las entradas en la forma `[784]`. Una posible forma de hacer esto sería aplicar una función de preprocesamiento a nuestro objeto `ClientData`.

In [4]:
def preprocess_dataset(dataset):
  """Create batches of 5 examples, and limit to 3 batches."""

  def map_fn(input):
    return collections.OrderedDict(
        x=tf.reshape(input['pixels'], shape=(-1, 784)),
        y=tf.cast(tf.reshape(input['label'], shape=(-1, 1)), tf.int64),
    )

  return dataset.batch(5).map(
      map_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE).take(5)


preprocessed_client_data = client_data.preprocess(preprocess_dataset)

# Notice that we have both reshaped and renamed the elements of the ordered dict.
first_client_dataset = preprocessed_client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


Quizas también queremos realizar un preprocesamiento más complejo (y posiblemente con estado), como por ejemplo aleatorizar.

In [5]:
def preprocess_and_shuffle(dataset):
  """Applies `preprocess_dataset` above and shuffles the result."""
  preprocessed = preprocess_dataset(dataset)
  return preprocessed.shuffle(buffer_size=5)

preprocessed_and_shuffled = client_data.preprocess(preprocess_and_shuffle)

# The type signature will remain the same, but the batches will be shuffled.
first_client_dataset = preprocessed_and_shuffled.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


## Interconectar con un `tff.Computation`

Ahora que podemos realizar algunas manipulaciones básicas con objetos `ClientData`, ya podemos ingresar datos en `tff.Computation`. Definamos un [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess) que implementa el [promedio federado](https://arxiv.org/abs/1602.05629) y exploremos los diferentes métodos para pasarle datos.

In [6]:
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
  ])
  return tff.learning.models.from_keras_model(
      model,
      # Note: input spec is the _batched_ shape, and includes the 
      # label tensor which will be passed to the loss function. This model is
      # therefore configured to accept data _after_ it has been preprocessed.
      input_spec=collections.OrderedDict(
          x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
          y=tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  
trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01))

Antes de empezar a trabajar con `IterativeProcess`, tenemos que hacer un comentario sobre la semántica de `ClientData`. Un objeto `ClientData` representa la *totalidad* de la población disponible para el entrenamiento federado, que en general [no está disponible para el entorno de ejecución de un sistema FL de producción](https://arxiv.org/abs/1902.01046) y es específico de la simulación. De hecho, `ClientData` brinda al usuario la capacidad de evitar por completo el cálculo federado y simplemente entrenar un modelo del lado del servidor como de costumbre a través de [`ClientData.create_tf_dataset_from_all_clients`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData?hl=en#create_tf_dataset_from_all_clients).

El entorno de simulación de TFF otorga al investigador el control total del bucle exterior. En particular, esto implica que el usuario o la secuencia de comandos del controlador de Python deben abordar las consideraciones de disponibilidad del cliente, abandono del cliente, etc. Por ejemplo, se podría modelar el abandono del cliente ajustando la distribución de muestreo en los `client_ids` de `ClientData's` de manera que la probabilidad de que se seleccionen los usuarios con más datos (y, en consecuencia, cálculos locales de mayor duración) sea menor.

Sin embargo, en un sistema federado real, quien entrena el modelo no puede seleccionar explícitamente a los clientes; la selección de clientes se delega al sistema que ejecuta el cálculo federado.

### Pasar `tf.data.Datasets` directamente a TFF

Una opción para interconectar entre `ClientData` y `IterativeProcess` es construir `tf.data.Datasets` en Python y pasar estos conjuntos de datos a TFF.

Tenga en cuenta que si usamos nuestros `ClientData` preprocesados, los conjuntos de datos que obtenemos son del tipo adecuado que necesita nuestro modelo definido anteriormente.

In [7]:
selected_client_ids = preprocessed_and_shuffled.client_ids[:10]

preprocessed_data_for_clients = [
    preprocessed_and_shuffled.create_tf_dataset_for_client(
        selected_client_ids[i]) for i in range(10)
]

state = trainer.initialize()
for _ in range(5):
  t1 = time.time()
  result = trainer.next(state, preprocessed_data_for_clients)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.934802532196045, round time 2.5420753955841064
loss 3.350963830947876, round time 0.45527172088623047
loss 3.1382687091827393, round time 0.47087883949279785
loss 3.0774152278900146, round time 0.4089682102203369
loss 2.9193594455718994, round time 0.3964221477508545


Sin embargo, si seguimos este camino, ***no podremos pasar trivialmente a la simulación multimáquina***. Los conjuntos de datos que construimos en el tiempo de ejecución local de TensorFlow pueden *capturar el estado del entorno Python que los envuelve* y no pueden con la serialización o deserialización cuando intentan hacer referencia a un estado que ya no está disponible para ellos. Esto puede manifestarse, por ejemplo, en el error inescrutable de `tensor_util.cc` de TensorFlow:

```
Check failed: DT_VARIANT == input.dtype() (21 vs. 20)
```

### Asignar la construcción y el preprocesamiento en los clientes

Para evitar este problema, TFF recomienda que sus usuarios consideren la creación de instancias y el preprocesamiento del conjunto de datos como *algo que sucede localmente en cada cliente* y que usen los ayudantes de TFF o `federated_map` para ejecutar explícitamente este código de preprocesamiento en cada cliente.

Conceptualmente, el motivo para preferir esto es claro: en el tiempo de ejecución local de TFF, los clientes solo tienen acceso "accidentalmente" al entorno global de Python debido al hecho de que toda la orquestación federada ocurre en una sola máquina. Vale la pena señalar en este punto que un pensamiento similar da lugar a la filosofía funcional, multiplataforma y siempre serializable de TFF.

TFF simplifica este cambio a través del atributo `dataset_computation` de `ClientData's`, un `tff.Computation` que toma un `client_id` y devuelve el `tf.data.Dataset` asociado.

Tenga en cuenta que `preprocess` simplemente funciona con `dataset_computation`; el atributo `dataset_computation` de `ClientData` preprocesado incorpora todo el proceso de preprocesamiento que acabamos de definir:

In [8]:
print('dataset computation without preprocessing:')
print(client_data.dataset_computation.type_signature)
print('\n')
print('dataset computation with preprocessing:')
print(preprocessed_and_shuffled.dataset_computation.type_signature)

dataset computation without preprocessing:
(string -> <label=int32,pixels=float32[28,28]>*)


dataset computation with preprocessing:
(string -> <x=float32[?,784],y=int64[?,1]>*)


Podríamos invocar `dataset_computation` y recibir un conjunto de datos eager en el tiempo de ejecución de Python, pero el verdadero poder de este enfoque se ejerce cuando componemos con un proceso iterativo u otro cálculo para evitar materializar estos conjuntos de datos en el tiempo de ejecución eager global. TFF proporciona una función ayudante [`tff.simulation.compose_dataset_computation_with_iterative_process`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/compose_dataset_computation_with_iterative_process) que se puede usa para hacer exactamente esto.

In [9]:
trainer_accepting_ids = tff.simulation.compose_dataset_computation_with_iterative_process(
    preprocessed_and_shuffled.dataset_computation, trainer)

Tanto este `tff.templates.IterativeProcesses` como el anterior se ejecutan de la misma manera; pero el primero acepta conjuntos de datos preprocesados de clientes ​​y el segundo acepta cadenas de texto que representan identificadores de clientes, controlando tanto la construcción del conjunto de datos como el preprocesamiento en su cuerpo. De hecho, el `state` se puede pasar entre los dos.

In [10]:
for _ in range(5):
  t1 = time.time()
  result = trainer_accepting_ids.next(state, selected_client_ids)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.6114611625671387, round time 1.4935951232910156
loss 2.612247943878174, round time 0.30751872062683105
loss 2.8368589878082275, round time 0.3043978214263916
loss 2.6863903999328613, round time 0.3107311725616455
loss 2.6816341876983643, round time 0.4325370788574219


### Escalar a un gran número de clientes

`trainer_accepting_ids` se puede usar inmediatamente en el tiempo de ejecución de múltiples máquinas de TFF y evita la materialización de `tf.data.Datasets` y del controlador (y, por lo tanto, serializarlos y enviarlos a los trabajadores).

Esto acelera significativamente las simulaciones distribuidas, especialmente con una gran cantidad de clientes, y permite la agregación intermedia para evitar una sobrecarga similar de serialización/deserialización.


### Análisis profundo opcional: componer de forma manual la lógica de preprocesamiento en TFF

TFF está diseñado para la composicionalidad desde cero; el tipo de composición que acaba de realizar el ayudante de TFF está totalmente bajo nuestro control como usuarios. Podríamos haber compuesto manualmente el cálculo de preprocesamiento que acabamos de definir con el `next` del entrenador de manera bastante simple:

In [11]:
selected_clients_type = tff.FederatedType(preprocessed_and_shuffled.dataset_computation.type_signature.parameter, tff.CLIENTS)

@tff.federated_computation(trainer.next.type_signature.parameter[0], selected_clients_type)
def new_next(server_state, selected_clients):
  preprocessed_data = tff.federated_map(preprocessed_and_shuffled.dataset_computation, selected_clients)
  return trainer.next(server_state, preprocessed_data)

manual_trainer_with_preprocessing = tff.templates.IterativeProcess(initialize_fn=trainer.initialize, next_fn=new_next)

De hecho, esto es efectivamente lo que el ayudante que usamos hace detrás de escena (además de realizar la verificación y manipulación de los tipos adecuados). Incluso podríamos haber expresado la misma lógica de manera un poco diferente, si serializamos `preprocess_and_shuffle` en `tff.Computation` y descomponemos `federated_map` en un paso que construye conjuntos de datos no preprocesados ​​y otro que ejecuta `preprocess_and_shuffle` en cada cliente.

Podemos verificar que esta forma más manual da como resultado cálculos con la misma signatura de tipo que el ayudante de TFF (nombres de parámetros de módulo):

In [12]:
print(trainer_accepting_ids.next.type_signature)
print(manual_trainer_with_preprocessing.next.type_signature)

(<state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,client_data={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,metrics=<distributor=<>,client_work=<train=<sparse_categorical_accuracy=float32,loss=float32,num_examples=int64,num_batches=int64>>,aggregator=<mean_value=<>,mean_weight=<>>,finalizer=<>>@SERVER>)
(<server_state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,selected_clients={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggre